**Brooklyn Crime** 

*Phases*
1) Data Mining
2) Data Exploration & Data Cleaning
3) Feature Engineering 
4) Predictive Modeling 
5) Visualization & Presentation

**1) Data Mining**

In [ ]:
#Importing Libraries
import pandas as pd
import geopandas as gpd
from geopandas import GeoDataFrame
from geopy.geocoders import Nominatim
import seaborn as sns 
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import geoplot as gplt

import math
from datetime import datetime
from shapely.geometry import Point
import csv
import os
from sklearn.impute import SimpleImputer
import contextily  as ctx

from datetime import date
from datetime import time
from sodapy import Socrata

import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster

from geopy.geocoders import Nominatim

from matplotlib import cm

import powerbiclient
from powerbiclient import Report, models


print("Libraries imported successfully...")

In [ ]:
####Loading data  from city of new york API
# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:


client = Socrata("data.cityofnewyork.us", None)
    
# Example authenticated client (needed for non-public datasets)
# client = Socrata(data.cityofnewyork.us,
#                  MyAppToken
#                  username=\"user@example.com\
#                  password=\"AFakePassword\")
    
# First specified number of results, returned as JSON from API 
# dictionaries by sodapy

##Removing missing  values in Complaints_df


    

In [ ]:
#COMPLAINTS
query = f"""
SELECT * 

WHERE boro_nm ="BROOKLYN"
    AND susp_race IS NOT NULL
    AND susp_sex IS NOT NULL
    AND susp_age_group IS NOT NULL
    AND cmplnt_to_dt  IS NOT NULL
    AND cmplnt_to_tm IS NOT NULL
    AND cmplnt_fr_dt IS NOT NULL
    AND cmplnt_fr_tm IS NOT NULL
    AND vic_age_group  IS NOT NULL
    AND cmplnt_fr_dt
BETWEEN '2022-01-01' AND '2022-12-31' 

LIMIT 100000

"""
    
Complaints = client.get("5uac-w243", query = query)
    
# Convert to pandas DataFrame 
Complaints_df = pd.DataFrame.from_records(Complaints)
#By far the largest df This program can use

In [ ]:
#Zillow
Zillow = pd.read_csv(r'D:\Motherless Brooklyn\Brooklyn-Crime\zillow NY for-sale properties.csv')
Zillow = Zillow.loc[Zillow.city.isin(['Brooklyn'])]

In [ ]:
#CENSUS
query = f"""
SELECT * 

WHERE borough ="Brooklyn"
    
LIMIT 100000

"""
Census = client.get("swpk-hqdp", query = query)
    
# Convert to pandas DataFrame 
Census = pd.DataFrame.from_records(Census)


**2) Data Cleaning and Data Exploration**

In [ ]:
#Columns list for Zillow

count = 0
for col in Zillow.columns:
    print(col)
    count+= 1

print("Number of columns are: ",count,)

In [ ]:
#Columns list for Complaints_df
count = 0
for col in Complaints_df.columns:
    print(col)
    count+= 1

print("Number of columns are: ",count,)

In [ ]:
#Columns list for Census
count = 0
for col in Census.columns:
    print(col)
    count+= 1

print("Number of columns are: ",count,)

        a) Data Cleaning

        i) Missing values

In [ ]:
# get the number of missing data points per column in Complaints_df
missing_values_count_Complaints = Complaints_df.isnull().sum()

# look at the # of missing points
missing_values_count_Complaints

##Relevant Null vallues dropped in mining via query. 

In [ ]:
#Adding weekdays
Complaints_df["cmplnt_fr_dt"]= Complaints_df['cmplnt_fr_dt'].astype('datetime64[ns]')
Complaints_df.dtypes
Complaints_df["Weekdays"] = Complaints_df['cmplnt_fr_dt'].dt.dayofweek
Complaints_df.loc[:,['cmplnt_fr_dt', 'Weekdays' ]]

In [ ]:
# get the number of missing data points per column in Zillow
missing_values_count_zillow = Zillow.isnull().sum()

# look at the # of missing points
missing_values_count_zillow


In [ ]:
# get the number of missing data points per column in Complaints_df
missing_values_count_Census = Census.isnull().sum()

# look at the # of missing points
missing_values_count_Census

    ii) Fixing Locations and precincts

In [ ]:
#Converting locations into real numbers for Zillow
dict_columns_type = {'longitude': float,
                'latitude': float
               }
               
   
Zillow = Zillow.astype(dict_columns_type)
Zillow.loc[:,['longitude', 'latitude']]

In [ ]:
#Converting locations into real numbers for Complaints_df

dict_columns_type = {'longitude': float,
                'latitude': float
               }
   
Complaints_df = Complaints_df.astype(dict_columns_type)
Complaints_df.loc[:,['longitude', 'latitude']]

    ii.a) GeoData

In [ ]:
#Precincts and Complaints_df
precincts_mp = gpd.read_file(r"D:\Motherless Brooklyn\Brooklyn-Crime\Police Precincts\geo_export_cd0a4750-fe4c-4415-9f05-3e7f0f353d91.shp")

precincts_mp.crs={'init': 'epsg:32630'}

Complaints_mp = gpd.GeoDataFrame(Complaints_df, geometry=gpd.points_from_xy(Complaints_df.longitude, Complaints_df.latitude))
Complaints_mp.crs = {'init': 'epsg:32630'}


precincts_mp = precincts_mp.loc[precincts_mp.precinct.isin([60.0,
    61.0,
    62.0,
    63.0,
    66.0,
    67.0,
    68.0,
    69.0,
    70.0,
    71.0,
    72.0,
    73.0,
    75.0,
    76.0,
    77.0,
    78.0,
    79.0,
    81.0,
    83.0,
    84.0,
    88.0,
    90.0,
    94.0
    ])].copy()

precincts_mp

In [ ]:
#Converting Zillow to a gpd
print("{}% of addresses were geocoded!".format(
    (1 - sum(np.isnan(Zillow["latitude"])) / len(Zillow)) * 100))

# Drop Places that were not successfully geocoded
Zillow = Zillow.loc[~np.isnan(Zillow["latitude"])]
Zillow_mp = gpd.GeoDataFrame(
    Zillow, geometry=gpd.points_from_xy(Zillow.longitude, Zillow.latitude))
Zillow_mp.crs = {'init': 'epsg:32630'}
Zillow_mp

In [ ]:
#Geocoding Census
geolocator = Nominatim(user_agent="Don")
def my_geocoder(row):
    try:
        point = geolocator.geocode(row).point
        return pd.Series({'latitude': point.latitude, 'longitude': point.longitude})
    except:
        return None

Census[['latitude', 'longitude']] = Census.apply(lambda x: my_geocoder(x['nta_name']), axis=1)

In [ ]:
#Converting Census to a gpd
print("{}% of addresses were geocoded!".format(
    (1 - sum(np.isnan(Census["latitude"])) / len(Census)) * 100))

# Drop Places that were not successfully geocoded
Census = Census.loc[~np.isnan(Census["latitude"])]
Census_mp = gpd.GeoDataFrame(
    Census, geometry=gpd.points_from_xy(Census.longitude, Census.latitude))
Census_mp.crs = {'init': 'epsg:32630'}
Census_mp.head()

    ii.b) Mapping

In [ ]:
# Create a map for Zillow_mp
ax = precincts_mp.plot(figsize=(8,8), color='whitesmoke', linestyle=':', edgecolor='black')
Zillow_mp.plot(markersize=1, ax=ax)

In [ ]:
# Create a map for Complaints_mp
ax = precincts_mp.plot(figsize=(8,8), color='whitesmoke', linestyle=':', edgecolor='black')
Complaints_mp.plot(markersize=1, ax=ax)

NB/= The locations with no records of complaints or properties are mostly parks

    ii.c) Merging with precincts_mp

In [ ]:
#Joining precinct with census
Census_mp = gpd.sjoin(Census_mp, precincts_mp )

In [ ]:
#Using spatial join to assign precincts to Neighborhood
Zillow_mp = gpd.sjoin(Zillow_mp, precincts_mp)


In [ ]:
#Dropping addr_pct_cd
Complaints_df.drop('addr_pct_cd', axis=1, inplace=True)

In [ ]:
#Replacing addr_pct_cd with precincts
Complaints_mp = gpd.sjoin(Complaints_mp, precincts_mp)

Complaints_mp.dtypes
#TODO: Separate categorical data and numeric data in Zillow_mp
##DONE

#TODO: Apply Neighborhood features to Complaints data.

In [ ]:
#Columns list for Complaints_mp
count = 0
for col in Complaints_mp.columns:
    print(col)
    count+= 1

print("Number of columns are: ",count,)

In [ ]:
#Columns list for Zillow_mp
count = 0
for col in Zillow_mp.columns:
    print(col)
    count+= 1

print("Number of columns are: ",count,)

In [ ]:
#Columns list for Census_mp
count = 0
for col in Census_mp.columns:
    print(col)
    count+= 1

print("Number of columns are: ",count,)

In [ ]:
#To datetime64
Complaints_mp['cmplnt_fr_tm'] = Complaints_mp['cmplnt_fr_tm'].astype('datetime64[ns]')
Complaints_mp['cmplnt_to_dt'] = pd.to_datetime(Complaints_mp['cmplnt_to_dt'])
Complaints_mp['cmplnt_to_tm'] = Complaints_mp['cmplnt_to_tm'].astype('datetime64[ns]')
Complaints_mp.loc[:,['cmplnt_fr_dt','cmplnt_fr_tm','cmplnt_to_tm','cmplnt_to_dt']]

In [ ]:
Complaints_mp.dtypes

    b) Data Exploration

    i) Complaints_mp

In [ ]:
Complaints_mp['cmplnt_dt'] = Complaints_mp.cmplnt_fr_dt.dt.date
Complaints_mp['cmplnt_tm'] = Complaints_mp.cmplnt_fr_tm.dt.hour
Complaints_mp.loc[:,['cmplnt_dt','cmplnt_tm']]
Complaints_mp.dtypes

In [ ]:
precincts_m1 = precincts_mp[["precinct", "geometry"]].set_index("precinct")
precincts_m1.head()

In [ ]:
# Number of crimes in each police district
plot_prenct = Complaints_mp.precinct.value_counts()
plot_prenct.head()
Complaints_mp.loc[:,['longitude','latitude']] 

In [ ]:
# Create a base map
Chloropleth_prenct = folium.Map(location=[40.634565,-73.928086], tiles='cartodbpositron', zoom_start=12)

# Add a choropleth map to the base map
Choropleth(geo_data=precincts_m1.__geo_interface__, 
           data=plot_prenct, 
           key_on="feature.id", 
           fill_color='YlGnBu', 
           legend_name='Major criminal incidents (Jan-Aug 2018)'
          ).add_to(Chloropleth_prenct)

# Display the map
Chloropleth_prenct

In [ ]:
# Create a base map
Heatmap_crime_density= folium.Map(location=[40.625,-73.95], tiles='cartodbpositron', zoom_start=12)

# Add a heatmap to the base map
HeatMap(data=Complaints_mp[['latitude', 'longitude']], radius=10).add_to(Heatmap_crime_density)

# Display the map
Heatmap_crime_density

In [ ]:
#Distribution of number of incidents per day
col = sns.color_palette()

plt.figure(figsize=(10, 6))
data = Complaints_mp.groupby('cmplnt_dt').count().iloc[:, 0]
sns.kdeplot(data=data, shade=True)
plt.axvline(x=data.median(), ymax=0.95, linestyle='--', color=col[1])
plt.annotate(
    'Median: ' + str(data.median()),
    xy=(data.median(), 0.004),
    xytext=(200, 0.005))
plt.title(
    'Distribution of number of incidents per day', fontdict={'fontsize': 16})
plt.xlabel('Incidents')
plt.ylabel('Density')
plt.legend().remove()
plt.show()

In [ ]:
#Incidents per hour
data = Complaints_mp.groupby('cmplnt_tm').count().iloc[:, 0]
data = data.reindex([
    0, 1, 2, 3, 4, 5,
    6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23
])

plt.figure(figsize=(10, 5))
with sns.axes_style("whitegrid"):
    ax = sns.barplot(
        data.index, (data.values / data.values.sum()) * 100,
        orient='v',
        palette=cm.ScalarMappable(cmap='Reds').to_rgba(data.values))

plt.title('Incidents per hour', fontdict={'fontsize': 16})
plt.xlabel('Hour')
plt.ylabel('Incidents (%)')

plt.show()

In [ ]:
#Incidents per Weekday
data = Complaints_mp.groupby('Weekdays').count().iloc[:, 0]
data = data.reindex([
    0, 1, 2, 3, 4, 5,
    6
])

plt.figure(figsize=(10, 5))
with sns.axes_style("whitegrid"):
    ax = sns.barplot(
        data.index, 
        (data.values / data.values.sum()) * 100,
        orient='v',
        palette=cm.ScalarMappable(cmap='Reds').to_rgba(data.values))

plt.title('Incidents per Weekday', fontdict={'fontsize': 16})
plt.xlabel('Weekday')
plt.ylabel('Incidents (%)')

plt.show()

In [ ]:
#Checking most popular crimes
data = Complaints_mp.groupby('ofns_desc').count().iloc[:, 0].sort_values(
    ascending=False)
data = data.reindex(np.append(np.delete(data.index, 1), 'OTHER OFFENSES'))

data

#TODO: Get real estate value for each precinct



In [ ]:
#lOCATION OF OCCURENCE
data = Complaints_mp.groupby('loc_of_occur_desc').count().iloc[:, 0]
data = data.reindex([
    'REAR OF', 'OPPOSITE OF', 'INSIDE', '(null)'
])

plt.figure(figsize=(10, 5))
with sns.axes_style("whitegrid"):
    ax = sns.barplot(
        data.index, 
        (data.values / data.values.sum()) * 100,
        orient='v',
        palette=cm.ScalarMappable(cmap='Reds').to_rgba(data.values))

plt.title('lOCATION OF OCCURENCE', fontdict={'fontsize': 16})
plt.xlabel('lOCATION')
plt.ylabel('Incidents (%)')

plt.show()

*Majority of the incidents happened inside*

In [ ]:
#Distribution by suspect race
data = Complaints_mp.groupby('susp_race').count().iloc[:, 0].sort_values(
    ascending=False)
data = data.reindex(np.append(np.delete(data.index, 1), 'OTHER'))

data

#TODO: Get real estate value for each precinct
##DONE



In [ ]:
#Distribution by suspect race visualization
data = Complaints_mp.groupby('susp_race').count().iloc[:, 0]
data = data.reindex([
    'BLACK', 'WHITE HISPANIC', 'WHITE', 'ASIAN / PACIFIC ISLANDER','BLACK HISPANIC','AMERICAN INDIAN/ALASKAN NATIVE','(null)'
])

plt.figure(figsize=(10, 5))
with sns.axes_style("whitegrid"):
    ax = sns.barplot(
        data.index, 
        (data.values / data.values.sum()) * 100,
        orient='v',
        palette=cm.ScalarMappable(cmap='Reds').to_rgba(data.values))

plt.title('Suspect race', fontdict={'fontsize': 16})
plt.xlabel('Suspects')
plt.ylabel('Incidents (%)')

plt.show()

In [ ]:
#Distribution by victim race visualization
data = Complaints_mp.groupby('vic_race').count().iloc[:, 0]
data = data.reindex([
    'BLACK', 'WHITE HISPANIC', 'WHITE', 'ASIAN / PACIFIC ISLANDER','BLACK HISPANIC','AMERICAN INDIAN/ALASKAN NATIVE','(null)'
])

plt.figure(figsize=(10, 5))
with sns.axes_style("whitegrid"):
    ax = sns.barplot(
        data.index, 
        (data.values / data.values.sum()) * 100,
        orient='v',
        palette=cm.ScalarMappable(cmap='Reds').to_rgba(data.values))

plt.title('Victim race', fontdict={'fontsize': 16})
plt.xlabel('Victim')
plt.ylabel('Incidents (%)')

plt.show()

In [ ]:
#Distribution by suspect sex visualization
data = Complaints_mp.groupby('susp_sex').count().iloc[:, 0]
data = data.reindex([
    'M', 'F', 'U','(null)'
])

plt.figure(figsize=(10, 5))
with sns.axes_style("whitegrid"):
    ax = sns.barplot(
        data.index, 
        (data.values / data.values.sum()) * 100,
        orient='v',
        palette=cm.ScalarMappable(cmap='Reds').to_rgba(data.values))

plt.title('Suspect sex', fontdict={'fontsize': 16})
plt.xlabel('Suspects')
plt.ylabel('Incidents (%)')

plt.show()

In [ ]:
#Distribution by Victim sex
data = Complaints_mp.groupby('vic_sex').count().iloc[:, 0].sort_values(
    ascending=False)
data = data.reindex(np.append(np.delete(data.index, 1), 'OTHER'))

data

#TODO:Get real estate value for each precinct



In [ ]:
#Distribution by victim sex visualization
data = Complaints_mp.groupby('vic_sex').count().iloc[:, 0]
data = data.reindex([
    'L', 'F', 'D','E'
])

plt.figure(figsize=(10, 5))
with sns.axes_style("whitegrid"):
    ax = sns.barplot(
        data.index, 
        (data.values / data.values.sum()) * 100,
        orient='v',
        palette=cm.ScalarMappable(cmap='Reds').to_rgba(data.values))

plt.title('Victim sex', fontdict={'fontsize': 16})
plt.xlabel('Victim')
plt.ylabel('Incidents (%)')

plt.show()

**Victim’s Sex Description**
D=Business/Organization, 
E=PSNY/People of the State of New York, 
F=Female, 
M=Male)

In [ ]:
precincts_m1 = precincts_mp[["precinct", "geometry"]].set_index("precinct")
plot_dict = Complaints_mp.precinct.value_counts()
plot_dict.to_frame(name="prenct_density")


In [ ]:
#Geographic Density of Different Crimes
crimes_cat = Complaints_mp['ofns_desc'].unique().tolist()

bk_land = precincts_mp.unary_union
bk_land = gpd.GeoDataFrame(gpd.GeoSeries(bk_land), crs={'init':'epsg:32630'})
bk_land = bk_land.rename(columns={0:'geometry'}).set_geometry('geometry')

fig, ax = plt.subplots(3, 3, sharex=True, sharey=True, figsize=(12,12))
for i , crime in enumerate(np.random.choice(crimes_cat, size=9, replace=False)):
    data = Complaints_mp.loc[Complaints_mp['ofns_desc'] == crime]
    ax = fig.add_subplot(3, 3, i+1)
    gplt.kdeplot(data,
                 shade=True,
                 shade_lowest=False,
                 clip = bk_land.geometry,
                 cmap='Reds',
                 ax=ax)
    gplt.polyplot(bk_land, ax=ax)
    ax.set_title(crime) 
plt.suptitle('Geographic Density of Different Crimes')
fig.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()


In [ ]:
#Average number of incidents per hour for various crimes
data = Complaints_mp.groupby(['cmplnt_tm', 'index_right', 'ofns_desc'],
                     as_index=False).count().iloc[:, :4]
data.rename(columns={'index_right': 'Incidents'}, inplace=True)
data = data.groupby(['cmplnt_tm', 'ofns_desc'], as_index=False).mean()
data = data.loc[data['ofns_desc'].isin(
    ['ROBBERY', 'KIDNAPPING', 'PETIT LARCENY', 'FELONY ASSAULT', 'POSSESSION OF STOLEN PROPERTY','CRIMINAL TRESPASS','GRAND LARCENY OF MOTOR VEHICLE'])]

sns.set_style("whitegrid")
fig, ax = plt.subplots(figsize=(14, 4))
ax = sns.lineplot(x='cmplnt_tm', y='Incidents', data=data, hue='ofns_desc')
ax.legend(loc='upper center', bbox_to_anchor=(0.5, 1.15), ncol=6)
plt.suptitle('Average number of incidents per hour')
fig.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

    ii) Zillow_mp

In [ ]:
Zillow_mp.dtypes

In [ ]:
#Separating categorical variables with numerical values. Location to be included in both.
Zillow_cat = Zillow_mp.loc[:,['city', 'state','longitude','latitude','land_space_unit', 'property_type','property_status','precinct', 'geometry']]
Zillow_num = Zillow_mp.loc[:,['price', 'bedroom_number', 'bathroom_number', 'price_per_unit', 'living_space', 'land_space']]
Zillow_num

In [ ]:
#distributions for all numeric variables 
for i in Zillow_num.columns:
    plt.hist(Zillow_num[i])
    plt.title(i)
    plt.show()

In [ ]:
#Zillow_num correlation
print(Zillow_num.corr())
sns.heatmap(Zillow_num.corr())

In [ ]:
Census_mp.dtypes
#Census_mp by population and merge with precincts_mp
##DONE

Zillow_mp.dtypes
#TODO:Group Zillow_mp by price_per_unit and merge with precincts_mp
##DONE

**3) Feature Engineering**

    a) Applying features to precincts_mp

In [ ]:
#Adding population to precincts_mp
Census_prenct = Census_mp.groupby('precinct').population.mean()
Census_prenct = Census_prenct.to_frame()
Census_prenct = Census_prenct.rename(columns= {1: 'population'})
precincts_mp = precincts_mp.merge(Census_prenct, on='precinct')
precincts_mp

In [ ]:
#Adding price_per_unit to precincts_mp
Zillow_prenct = Zillow_mp.groupby('precinct').price_per_unit.mean()
Zillow_prenct = Zillow_prenct.to_frame()
Zillow_prenct = Zillow_prenct.rename(columns= {1: 'price_per_unit'})
precincts_mp = precincts_mp.merge(Zillow_prenct, on='precinct')

precincts_mp

    a) i)Analysis on precincts (price per unit & population)